<a href="https://colab.research.google.com/github/konapalask/NLP_Projects/blob/main/Text%20extraction%20from%20pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber sentence-transformers chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 112.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.7 MB/s eta 0

In [ ]:
import pdfplumber

# ✅ Upload your PDF to Colab (Attention Is All You Need)
from google.colab import files
uploaded = files.upload()

# Extract Text
pdf_file = list(uploaded.keys())[0]
full_text = ""
with pdfplumber.open(pdf_file) as pdf:
    for page in pdf.pages:
        full_text += page.extract_text() + "\n"

# Clean Text
clean_text = "\n".join([line.strip() for line in full_text.split('\n') if line.strip() != ""])

# Preview cleaned text
print(clean_text[:10000000000])


Saving B Pardhin Resume.pdf to B Pardhin Resume.pdf
BURAGADDA PARDHIN SIVA SAI pardhin.sai@gmail.com
+91 9063508918
Software Tester Vijayawada, 521108.
I am a dedicated software tester with a strong focus on ensuring quality and reliability in software products.
Skilled in identifying bugs, improving system performance, and delivering seamless user experiences.
TECHNICAL SKILLS MANUAL TESTING SKILLS SUMMARY
Automation Testing Proficient in creating and running test cases to check if software
Manual Testing works correctly.
Java and HTML Experienced in testing features, fixing bugs, and making sure
SDLC and STLC updates don’t break anything.
Agile Methodology Skilled in finding and reporting issues using tools like JIRA.
TestNG, Apache Poi and Familiar with the process of developing and testing software,
Cucumber especially in Agile teams.
API Testing Strong at solving problems and working well with developers to
improve quality.
TOOLS AUTOMATION TESTING SKILLS SUMMARY
JIRA Experienced 

In [ ]:
import re

def split_into_chunks(text, chunk_size=300, overlap=50):
    words = re.split(r'\s+', text)
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk:
            chunks.append(chunk)
    return chunks

chunks = split_into_chunks(clean_text)

print(f"\n✅ Total Chunks Created: {len(chunks)}")
print("\n🔍 Sample Chunk Preview:\n")
print(chunks[0][:500])



✅ Total Chunks Created: 3

🔍 Sample Chunk Preview:

BURAGADDA PARDHIN SIVA SAI pardhin.sai@gmail.com +91 9063508918 Software Tester Vijayawada, 521108. I am a dedicated software tester with a strong focus on ensuring quality and reliability in software products. Skilled in identifying bugs, improving system performance, and delivering seamless user experiences. TECHNICAL SKILLS MANUAL TESTING SKILLS SUMMARY Automation Testing Proficient in creating and running test cases to check if software Manual Testing works correctly. Java and HTML Experienc


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

print(f"\n✅ Embedding Vector Example (Shape): {embeddings[0].shape}")



✅ Embedding Vector Example (Shape): (384,)


In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(Settings(anonymized_telemetry=False))

collection = client.create_collection(name="attention_paper")

# Insert into DB
for idx, (chunk, embedding) in enumerate(zip(chunks, embeddings)):
    collection.add(
        ids=[f"chunk_{idx}"],
        embeddings=[embedding.tolist()],
        documents=[chunk],
        metadatas=[{"chunk_id": idx, "source": pdf_file}]
    )

# ✅ Verify Insertion
results = collection.peek()
print("\n✅ Sample from ChromaDB:\n")
print(results)



✅ Sample from ChromaDB:

{'ids': ['chunk_0', 'chunk_1', 'chunk_2'], 'embeddings': array([[-0.0684513 ,  0.0028301 , -0.06043512, ..., -0.0047248 ,
        -0.01198437,  0.04666923],
       [-0.11192004,  0.00038622, -0.10618552, ..., -0.05031322,
        -0.00252837, -0.00773597],
       [-0.01800849,  0.01747497, -0.04270579, ...,  0.0398354 ,
         0.04503404, -0.00436975]]), 'documents': ['BURAGADDA PARDHIN SIVA SAI pardhin.sai@gmail.com +91 9063508918 Software Tester Vijayawada, 521108. I am a dedicated software tester with a strong focus on ensuring quality and reliability in software products. Skilled in identifying bugs, improving system performance, and delivering seamless user experiences. TECHNICAL SKILLS MANUAL TESTING SKILLS SUMMARY Automation Testing Proficient in creating and running test cases to check if software Manual Testing works correctly. Java and HTML Experienced in testing features, fixing bugs, and making sure SDLC and STLC updates don’t break anything. Agi